## Clase 3

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df_genres = pd.read_csv('./data/movie_genres.csv')
df_ratings = pd.read_csv('./data/ratings_train.csv')

### Creando un perfil

In [3]:
list_of_genres = df_genres.genre.unique().tolist()
list_of_genres.sort()
df_genres['value'] = 1
df_genre_matrix = df_genres.pivot(index='movieID', columns='genre', values='value').fillna(0).rename_axis(columns = None).reset_index()
df_genre_matrix.head()

,movieID,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_rating_genre = df_genre_matrix.merge(df_ratings, left_on='movieID', right_on='movieID')
df_rating_genre.head()

,movieID,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Western,ID,userID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,234,35886,5.0,15,10,1997,7,55,48
1,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,377,9117,5.0,10,11,1997,16,28,49
2,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,761,23172,5.0,8,1,1998,19,5,24
3,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1002,18059,4.0,7,4,1998,20,9,36
4,1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,2365,67068,3.0,30,9,1998,22,36,24


In [5]:
list_of_profiles = []
for a_user in tqdm(df_ratings.userID.unique()):
    profile_by_user = dict()
    profile_by_user['userID'] = a_user
    df_user = df_rating_genre[df_rating_genre.userID == a_user]
    
    for a_genre in list_of_genres:
        profile_by_user[a_genre] = df_user.loc[df_user[a_genre] == 1, "rating"].mean() / 5.0
    
    list_of_profiles.append(profile_by_user)
    

100%|██████████| 2113/2113 [00:14<00:00, 146.02it/s]


In [6]:
df_profile = pd.DataFrame.from_dict(list_of_profiles).fillna(0)
df_profile.head()

,userID,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,51198,0.783824,0.798000,0.788889,0.766667,0.779000,0.865116,0.883333,0.833028,0.796667,...,0.728571,0.9,0.840000,0.804762,0.810811,0.784211,0.0,0.811111,0.816667,0.733333
1,29202,0.512500,0.503182,0.447059,0.433333,0.443613,0.534911,0.653333,0.566725,0.492742,...,0.464516,0.6,0.504545,0.548750,0.536408,0.509357,0.0,0.531180,0.575556,0.556522
2,32283,0.687004,0.694545,0.806452,0.758333,0.694751,0.728571,0.828571,0.739817,0.694805,...,0.662162,0.8,0.700000,0.742857,0.723832,0.656566,0.0,0.701917,0.721212,0.769565
3,36495,0.673034,0.677551,0.763636,0.742857,0.764198,0.767241,0.500000,0.755914,0.631579,...,0.800000,0.0,0.800000,0.725000,0.740541,0.640000,0.0,0.709709,0.762500,0.600000
4,28350,0.653191,0.666667,0.705769,0.658915,0.632463,0.635714,0.000000,0.669822,0.656180,...,0.429412,0.8,0.641667,0.615000,0.670940,0.597561,0.0,0.634375,0.684211,0.635294


In [7]:
df_profile.to_csv('./data/profile_by_user.csv', index=False)